In [9]:
from pprint import pprint
from functools import reduce
import brotli
import pickle


with open("./dump/dbeaver_dbeaver/commits.szhat".replace("\\","/"), "rb") as handler:
    # print(handler.read())
    data = handler.read()
    decompressed_data = brotli.decompress(data)
    obj = pickle.loads(decompressed_data)

# print(c[1].files)
# avg = reduce(lambda x, y: x+y, map(lambda c: c.files, commits))/ len(commits)
# c = commits[0]

# print(c.stats.additions)

In [10]:
print([o for o in obj if o.files is None])

[Commit(sha="fffdc7e483462617869d63f38d4f6473b084c876")]


## Gathering the data

In [7]:
import glob
import pickle
from collections import defaultdict
from pprint import pprint
import pandas as pd

repo_dicts = defaultdict(lambda: {})
metric_dicts = defaultdict(lambda: defaultdict(lambda: {}))
for p in glob.glob(".\intermediate\*\*.pickle"):
    with open(p, "rb") as file:
        metric = pickle.load(file)
        dic = metric.__dict__
        repo_name = dic.pop("repo_name")
        metric_type = p.split('\\')[-1].split(".")[0]

        metric_dicts[metric_type][repo_name] |= dic
        repo_dicts[repo_name] |= dic
        
data = pd.DataFrame.from_dict(repo_dicts)
data.to_csv("metrics.csv",index=True)

## Working with the data

In [10]:
name_predicate_pairs= [
    ("Have releases", lambda inner: inner.get("[Releases] Count")),
    ("Have no releases", lambda inner: not inner.get("[Releases] Count")),
    ("Have source code only releases", lambda inner: not inner.get("[Releases] Total downloads")),
    ("Have binary and source code releases", lambda inner: inner.get("[Releases] Total downloads")),
    ]


data_to_gather = {
    name: {repo: inner for repo, inner in repo_dicts.items() if func(inner)}
    for name, func in name_predicate_pairs
}

print(f"Metrics count: {len(data)}")
print(f"Repo    count: {len(data.keys())}\n")

for type, inner in data_to_gather.items():
    print(f"{type}: {len(inner)}")
    # display(pd.DataFrame(inner))

Metrics count: 107
Repo    count: 173

Have releases: 121
Have no releases: 52
Have source code only releases: 114
Have binary and source code releases: 59


## Calculating correlations

In [129]:
from itertools import combinations_with_replacement
from IPython.display import display, Markdown, Latex

LOWER_BOUND = 0.65

def prune_dict(dct, mask):
    result = {}
    for k, v in mask.items():
        if isinstance(v, dict):
            value = prune_dict(dct[k], v)
            if value: # check that dict is non-empty
                result[k] = value
        elif v:
            result[k] = dct[k]
    return result

# for data, name in zip([data, downloads_data, no_downloads_data], ["correlations", "downloads_correlations", "no_downloads_correlations"]):
values = {name:data.loc[name] for name in data.index}
correlations = pd.DataFrame(values).corr(method='pearson').values
frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)
frame.to_csv('correlations.csv', index=True)


for g1, g2 in combinations_with_replacement(metric_dicts.items(), 2):
    m1, inner_dict1 = g1
    m2, inner_dict2 = g2

    res = {name:inner_dict1[name] | inner_dict2[name] for name in inner_dict1}
    res_df = pd.DataFrame(res).transpose().corr(method='pearson')
    metric1_names = list(list(inner_dict1.values())[0].keys())
    metric2_names = list(list(inner_dict2.values())[0].keys())
    df = res_df[metric1_names].loc[metric2_names]
    
    # df = df[df[df.columns] > 0.7]
    mask = df.where(df != 1).where(df != 0).where(abs(df) > LOWER_BOUND)
    mask = mask.dropna(how="all").dropna(axis="columns", how="all")
    mask = mask.iloc[:, :-1].notnull().astype(float).groupby(level=0).sum().to_dict()
    tmp = prune_dict(df.to_dict(), mask)

    if tmp:
        display(Markdown(f"# {m1} X {m2}"))
        for name, val in tmp.items():   
            # print(tmp)
            display(pd.DataFrame({name: val}))
    # display(pd.DataFrame(df))

C:\Users\79134\AppData\Local\Temp/ipykernel_79028/3672673628.py:20: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)


# Commits X Commits

,[Commits] Count
[Commits] Per day,0.784588


,[Commits] Total lines added
[Commits] Average additions,0.705340
[Commits] Average deletions,0.802878
[Commits] Total lines deleted,0.958184


,[Commits] Total lines deleted
[Commits] Average deletions,0.673539
[Commits] Total lines added,0.958184


...

,[Stars] Count
[Stars] Per day (True),0.673777


# WorkflowRuns X WorkflowRuns

,[Workflow Runs] Count
[Workflow Runs] Average successes per day,0.893371
[Workflow Runs] Average successes per day (True),0.831688


,[Workflow Runs] Average successes per day
[Workflow Runs] Average successes per day (True),0.850415
[Workflow Runs] Count,0.893371


,[Workflow Runs] Average successes per day (True)
[Workflow Runs] Average fails per day (True),0.659509
[Workflow Runs] Average successes per day,0.850415
[Workflow Runs] Count,0.831688
